# Prompt Test Notebook

This notebook is isolated for experimenting with channel system prompts and seeing the resulting messages.


In [1]:
import os
import json
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

from main import load_signs, generate_daily_context, build_channel_sign_messages

root_dir = Path.cwd()
if not (root_dir / ".env").exists() and (root_dir.parent / ".env").exists():
    root_dir = root_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
RSS_FEED_URL = os.getenv("RSS_FEED_URL")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
TELEGRAM_API_ID = os.getenv("TELEGRAM_API_ID")
TELEGRAM_API_HASH = os.getenv("TELEGRAM_API_HASH")
TELEGRAM_NEWS_CHANNEL = os.getenv("TELEGRAM_NEWS_CHANNEL")
TELEGRAM_NEWS_LIMIT = int(os.getenv("TELEGRAM_NEWS_LIMIT", "20"))
TELETHON_SESSION = os.getenv("TELETHON_SESSION", "telethon.session")
TELETHON_SESSION_STRING = os.getenv("TELETHON_SESSION_STRING")

if not OPENAI_API_KEY:
    raise RuntimeError("Set OPENAI_API_KEY in .env")

client = OpenAI(api_key=OPENAI_API_KEY)
signs = load_signs()
telegram_source = {
    "api_id": int(TELEGRAM_API_ID) if TELEGRAM_API_ID else None,
    "api_hash": TELEGRAM_API_HASH,
    "channel": TELEGRAM_NEWS_CHANNEL,
    "limit": TELEGRAM_NEWS_LIMIT,
    "session_path": TELETHON_SESSION,
    "session_string": TELETHON_SESSION_STRING,
}


In [36]:
# Full pipeline: generate vibes + post-process global summary + preview channel messages
from main import fetch_telegram_messages

# --- SYSTEM PROMPT (edit here) ---
channel_system_prompt = (
    "Ти — редакторка українського Telegram-каналу з щоденними зодіак-оновленнями, "
    "які обігрують українські новини без псевдоастрології.\n\n"
    "Тон: інтелігентна іронія + підтримка.\n"
    "Стиль: коротко, влучно, телеграмно.\n\n"
    "Фокус: гроші, кохання, дружба\n\n"
    "ЗАБОРОНЕНО:\n"
    "- будь-яка псевдоастрологія (планети, ретроградність, \u201cзірки кажуть\u201d)\n"
    "- висміювання трагедій, жертв, обстрілів, втрат\n"
    "- токсичний або зневажливий гумор\n"
    "- конкретні неперевірені факти, інсайди, назви телеграм-каналів\n"
    "- поради, що можуть нашкодити (медицина, безпека, фінанси)\n\n"
    "ОСНОВНЕ ПРАВИЛО:\n"
    "Ти не передбачаєш майбутнє. Ти описуєш, "
    "як різні типи людей проживають сьогоднішній інформаційний день."
)

# --- STEP 1: Fetch news ---
messages = await fetch_telegram_messages(
    api_id=telegram_source.get("api_id"),
    api_hash=telegram_source.get("api_hash"),
    channel=telegram_source.get("channel"),
    limit=telegram_source.get("limit", 20),
    session_path=telegram_source.get("session_path", "telethon.session"),
    session_string=telegram_source.get("session_string"),
)
news_blob = "\n".join([f"- {m}" for m in messages]) or "Немає налаштованого джерела новин."

signs_payload = {
    sign: {
        "traits": data.get("traits", []),
        "specificity": data.get("specificity", ""),
    }
    for sign, data in signs.items()
}

# --- STEP 2: Generate vibes + raw global summary ---
user_prompt = (
    "Повідомлення з джерела новин:\n"
    f"{news_blob}\n\n"
    "Конфіг знаків:\n"
    f"{json.dumps(signs_payload, ensure_ascii=False)}\n\n"
    "Поверни JSON з ключами: affirmation (коротке 1 речення), "
    "global_summary (рядок) і vibes (об\u2019єкт: знак -> текст вайбу). "
    "Лише JSON."
)

response = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {"role": "system", "content": channel_system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    response_format={"type": "json_object"},
    temperature=0.9,
)

payload = json.loads(response.choices[0].message.content)

# --- STEP 3: Post-process global summary ---
raw_global_summary = payload.get("global_summary", "")

summary_user_prompt = (
    "Напиши коротке інтро до щоденних зодіак-прогнозів у Telegram-каналі. "
    "Це НЕ дайджест новин, а настрій дня перед прогнозами. "
    "Візьми 1 факт із новин і обіграй його непрямо, без слова 'НОВИНИ'. "
    "Рівно 1 речення, до 140 символів. "
    "Без шаблонного оптимізму. "
    "\n\n"
    f"{news_blob}\n\n"
    f"{raw_global_summary}"
)


summary_response = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {"role": "system", "content": channel_system_prompt},
        {"role": "user", "content": summary_user_prompt},
    ],
    temperature=0.9,
)

polished_summary = summary_response.choices[0].message.content.strip()

# --- STEP 4: Build final channel messages ---
context_final = {
    "affirmation": payload.get("affirmation", ""),
    "global_summary": polished_summary,
    "vibes": payload.get("vibes", {}),
}

final_messages = build_channel_sign_messages(context_final, signs)
for msg in final_messages:
    print(msg)
    print("---")


Сьогодні об'єднуємо зусилля, щоб підтримати один одного в усіх випробуваннях.
Сьогоднішній день знову показує, що разом нам під силу більше. А вугільні шахти знову працюють — маленька перемога на нашій дорозі.

♈ Овен: Сьогодні твоє імпульсивне прагнення може призвести до сміливих рішень – використай його мудро.
---
♉ Телець: Стабільність і терпіння сьогодні принесуть плоди, тримайся наміток.
---
♊ Близнюки: Допитливість допоможе знайти нові рішення – спілкуйся й адаптуйся.
---
♋ Рак: Твоя чутливість стане опорою для близьких, будь тією підтримкою, якої вони потребують.
---
♌ Лев: Твоя впевненість може надихати інших, не бійтеся ділитися своєю енергією.
---
♍ Діва: Аналізуй ситуацію та шукай практичні рішення, твоя увага до деталей важлива.
---
♎ Терези: Сьогодні важливо підтримувати гармонію в стосунках, дипломатія допоможе уникнути конфліктів.
---
♏ Скорпіон: Твоя стратегічна інтуїція підкаже, як діяти в складних ситуаціях – довіряй своїм відчуттям.
---
♐ Стрілець: Оптимізм і авантюр